# Anti Money Laundering Detection with GNN node classification
### This notenook includes GNN model training and dataset implementation with PyG library. In this example, we used LI-Small_Trans.csv as our dataset for training and testing.  
### For more details, please view https://github.com/issacchan26/AntiMoneyLaunderingDetectionWithGNN

In [ ]:
import datetime
import os
from typing import Callable, Optional
import pandas as pd
from sklearn import preprocessing
import numpy as np
import torch

from torch_geometric.data import (
    Data,
    InMemoryDataset
)

pd.set_option('display.max_columns', None) # Show all columns when print dataframe
path = './data/raw/LI-Small_Trans.csv'
df = pd.read_csv(path)

# Data visualization and possible feature engineering
Let's look into the dataset

In [25]:
print(df.head())

          Timestamp  From Bank    Account  To Bank  Account.1  \
0  2022/09/01 00:08         11  8000ECA90       11  8000ECA90   
1  2022/09/01 00:21       3402  80021DAD0     3402  80021DAD0   
2  2022/09/01 00:00         11  8000ECA90     1120  8006AA910   
3  2022/09/01 00:16       3814  8006AD080     3814  8006AD080   
4  2022/09/01 00:00         20  8006AD530       20  8006AD530   

   Amount Received Receiving Currency  Amount Paid Payment Currency  \
0       3195403.00          US Dollar   3195403.00        US Dollar   
1          1858.96          US Dollar      1858.96        US Dollar   
2        592571.00          US Dollar    592571.00        US Dollar   
3            12.32          US Dollar        12.32        US Dollar   
4          2941.56          US Dollar      2941.56        US Dollar   

  Payment Format  Is Laundering  
0   Reinvestment              0  
1   Reinvestment              0  
2         Cheque              0  
3   Reinvestment              0  
4   Reinvest

After the viewing the dataframe, we suggest that we can extract all accounts from receiver and payer among all transcation for sorting the suspicious accounts. We can transform the whole dataset into node classification problem by considering accounts as nodes while transcation as edges.

The object columns should be encoded into classes with sklearn LabelEncoder.

In [26]:
print(df.dtypes)

Timestamp              object
From Bank               int64
Account                object
To Bank                 int64
Account.1              object
Amount Received       float64
Receiving Currency     object
Amount Paid           float64
Payment Currency       object
Payment Format         object
Is Laundering           int64
dtype: object


Check if there are any null values

In [27]:
print(df.isnull().sum())

Timestamp             0
From Bank             0
Account               0
To Bank               0
Account.1             0
Amount Received       0
Receiving Currency    0
Amount Paid           0
Payment Currency      0
Payment Format        0
Is Laundering         0
dtype: int64


There are two columns representing paid and received amount of each transcation, wondering if it is necessary to split the amount into two columns when they shared the same value, unless there are transcation fee/transcation between different currency. Let's find out 

In [28]:
print('Amount Received equals to Amount Paid:')
print(df['Amount Received'].equals(df['Amount Paid']))
print('Receiving Currency equals to Payment Currency:')
print(df['Receiving Currency'].equals(df['Payment Currency']))

Amount Received equals to Amount Paid:
False
Receiving Currency equals to Payment Currency:
False


It seens involved the transcations between different currency, let's print it out

In [29]:
not_equal1 = df.loc[~(df['Amount Received'] == df['Amount Paid'])]
not_equal2 = df.loc[~(df['Receiving Currency'] == df['Payment Currency'])]
print(not_equal1)
print('---------------------------------------------------------------------------')
print(not_equal2)

                Timestamp  From Bank    Account  To Bank  Account.1  \
2770     2022/09/01 00:12        394  80056EDE0      394  80056EDE0   
8081     2022/09/01 00:28      11701  800C95BF0    11701  800C95BF0   
10451    2022/09/01 00:18      22481  80105E630    22481  80105E630   
12948    2022/09/01 00:17       1439  8014545C0     1439  8014545C0   
13799    2022/09/01 00:02         20  8015D68E0       20  8015D68E0   
...                   ...        ...        ...      ...        ...   
6924007  2022/09/10 23:57       9096  80356BD61     9096  80356BD60   
6924009  2022/09/10 23:30       9096  80356BD61     9096  80356BD60   
6924019  2022/09/10 23:38      13474  803A93631    13474  803A93630   
6924021  2022/09/10 23:31      13474  803A93631    13474  803A93630   
6924023  2022/09/10 23:56      13474  803A93631    13474  803A93630   

         Amount Received Receiving Currency  Amount Paid Payment Currency  \
2770           47.610000               Euro        55.79        US Dol

The size of two df shows that there are transcation fee and transcation between different currency, we cannot combine/drop the amount columns.

As we are going to encode the columns, we have to make sure that the classes of same attribute are aligned.
Let's check if the list of Receiving Currency and Payment Currency are the same

In [30]:
print(sorted(df['Receiving Currency'].unique()))
print(sorted(df['Payment Currency'].unique()))

['Australian Dollar', 'Bitcoin', 'Brazil Real', 'Canadian Dollar', 'Euro', 'Mexican Peso', 'Ruble', 'Rupee', 'Saudi Riyal', 'Shekel', 'Swiss Franc', 'UK Pound', 'US Dollar', 'Yen', 'Yuan']
['Australian Dollar', 'Bitcoin', 'Brazil Real', 'Canadian Dollar', 'Euro', 'Mexican Peso', 'Ruble', 'Rupee', 'Saudi Riyal', 'Shekel', 'Swiss Franc', 'UK Pound', 'US Dollar', 'Yen', 'Yuan']


# Data Preprocessing
### We will show the functions used in the PyG dataset first, dataset and model training will be provided in bottom section

In the data preprocessing, we perform below transformation:  
1. Transform the Timestamp with min max normalization.  
2. Create unique ID for each account by adding bank code with account number.  
3. Create receiving_df with the information of receiving accounts, received amount and currency
4. Create paying_df with the information of payer accounts, paid amount and currency
5. Create a list of currency used among all transactions
6. Label the 'Payment Format', 'Payment Currency', 'Receiving Currency' by classes with sklearn LabelEncoder


In [ ]:
# ฟังก์ชันสำหรับเข้ารหัส (Label Encoding) ค่าที่เป็นข้อความ (categorical columns)
def df_label_encoder(df, columns):
    # สร้างอ็อบเจกต์ LabelEncoder จาก sklearn
    le = preprocessing.LabelEncoder()

    # วนลูปเข้ารหัสทุกคอลัมน์ที่ระบุ
    for i in columns:
        # แปลงค่าทั้งหมดในคอลัมน์เป็น string แล้วเข้ารหัสเป็นตัวเลข
        df[i] = le.fit_transform(df[i].astype(str))
    
    # คืนค่า DataFrame หลังจากเข้ารหัสแล้ว
    return df


# ฟังก์ชันสำหรับเตรียมข้อมูลก่อนนำเข้าโมเดล (Data Preprocessing)
def preprocess(df):
    # 1 เข้ารหัสคอลัมน์ที่เป็นข้อความ (Payment Format, Payment Currency, Receiving Currency)
    df = df_label_encoder(df, ['Payment Format', 'Payment Currency', 'Receiving Currency'])

    # 2️ แปลงคอลัมน์ Timestamp จาก string → datetime object
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])

    # 3️ แปลง Timestamp เป็นค่าตัวเลข (จำนวน nanoseconds นับจาก epoch)
    df['Timestamp'] = df['Timestamp'].apply(lambda x: x.value)

    # 4️ ทำการ Normalize ค่า Timestamp ให้อยู่ในช่วง [0, 1]
    df['Timestamp'] = (df['Timestamp'] - df['Timestamp'].min()) / (df['Timestamp'].max() - df['Timestamp'].min())

    # 5️ รวมรหัสธนาคารกับหมายเลขบัญชีเพื่อสร้างรหัสบัญชีที่ไม่ซ้ำ (unique account ID)
    # เช่น "11" (From Bank) + "_" + "8000ECA90" (Account) → "11_8000ECA90"
    df['Account'] = df['From Bank'].astype(str) + '_' + df['Account']
    df['Account.1'] = df['To Bank'].astype(str) + '_' + df['Account.1']

    # 6️ เรียงลำดับข้อมูลตามชื่อบัญชี เพื่อให้ข้อมูลมีลำดับที่คงที่
    df = df.sort_values(by=['Account'])

    # 7️ สร้าง DataFrame ของบัญชีผู้รับ (receiving_df)
    # มีข้อมูล: บัญชีผู้รับ, จำนวนเงินที่ได้รับ, และรหัสสกุลเงิน
    receiving_df = df[['Account.1', 'Amount Received', 'Receiving Currency']]

    # 8️ สร้าง DataFrame ของบัญชีผู้จ่าย (paying_df)
    # มีข้อมูล: บัญชีผู้จ่าย, จำนวนเงินที่จ่าย, และรหัสสกุลเงิน
    paying_df = df[['Account', 'Amount Paid', 'Payment Currency']]

    # 9️ เปลี่ยนชื่อคอลัมน์ Account.1 → Account เพื่อให้ชื่อสอดคล้องกับฝั่งผู้จ่าย
    receiving_df = receiving_df.rename({'Account.1': 'Account'}, axis=1)

    #  ดึงรายการสกุลเงินทั้งหมดที่ปรากฏในข้อมูล และจัดเรียงให้อยู่ในลำดับคงที่
    currency_ls = sorted(df['Receiving Currency'].unique())

    return df, receiving_df, paying_df, currency_ls


Let's have a look of processed df

In [32]:
df, receiving_df, paying_df, currency_ls = preprocess(df = df)
print(df.head())

         Timestamp  From Bank      Account  To Bank        Account.1  \
3408783   0.266147          0  0_800060CE0    11314  11314_800990320   
3986981   0.318925          0  0_800060CE0    11314  11314_800990320   
4804475   0.393400          0  0_800060CE0    11314  11314_800990320   
4804474   0.394151          0  0_800060CE0    11314  11314_800990320   
6690464   0.547730          0  0_800060CE0     1390   1390_800E49870   

         Amount Received  Receiving Currency  Amount Paid  Payment Currency  \
3408783          8081.58                   4      8081.58                 4   
3986981         47468.31                   4     47468.31                 4   
4804475          8081.58                   4      8081.58                 4   
4804474         47468.31                   4     47468.31                 4   
6690464           787.72                   4       787.72                 4   

         Payment Format  Is Laundering  
3408783               4              0  
3986981   

paying df and receiving df:

In [33]:
print(receiving_df.head())
print(paying_df.head())

                 Account  Amount Received  Receiving Currency
3408783  11314_800990320          8081.58                   4
3986981  11314_800990320         47468.31                   4
4804475  11314_800990320          8081.58                   4
4804474  11314_800990320         47468.31                   4
6690464   1390_800E49870           787.72                   4
             Account  Amount Paid  Payment Currency
3408783  0_800060CE0      8081.58                 4
3986981  0_800060CE0     47468.31                 4
4804475  0_800060CE0      8081.58                 4
4804474  0_800060CE0     47468.31                 4
6690464  0_800060CE0       787.72                 4


currency_ls:

In [34]:
print(currency_ls)

[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14)]


We would like to extract all unique accounts from payer and receiver as node of our graph. It includes the unique account ID, Bank code and the label of 'Is Laundering'.  
In this section, we consider both payer and receiver involved in a illicit transaction as suspicious accounts, we will label both accounts with 'Is Laundering' == 1.

In [ ]:
def get_all_account(df):
    """
    ฟังก์ชันนี้มีหน้าที่รวบรวมบัญชีทั้งหมดจากทั้งฝั่งผู้โอน (payer) และผู้รับ (receiver)
    เพื่อสร้างรายการบัญชีที่ใช้เป็นโหนด (nodes) ในกราฟของระบบตรวจจับการฟอกเงิน
    พร้อมกำหนดค่าป้ายกำกับ (label) ให้แต่ละบัญชีว่าเป็นบัญชีปกติ (0) หรือเป็นบัญชีที่เกี่ยวข้องกับการฟอกเงิน (1)
    """
    
    #ดึงเฉพาะข้อมูลฝั่งผู้โอน (payer) ประกอบด้วย Account และรหัสธนาคารต้นทาง
    ldf = df[['Account', 'From Bank']]

    # ดึงเฉพาะข้อมูลฝั่งผู้รับ (receiver) ประกอบด้วย Account.1 และรหัสธนาคารปลายทาง
    rdf = df[['Account.1', 'To Bank']]

    # เลือกเฉพาะธุรกรรมที่มีการฟอกเงิน (Is Laundering == 1)
    suspicious = df[df['Is Laundering'] == 1]

    # ดึงบัญชีฝั่งผู้โอนที่อยู่ในธุรกรรมฟอกเงิน
    s1 = suspicious[['Account', 'Is Laundering']]

    # ดึงบัญชีฝั่งผู้รับที่อยู่ในธุรกรรมฟอกเงิน
    s2 = suspicious[['Account.1', 'Is Laundering']]

    # เปลี่ยนชื่อคอลัมน์ 'Account.1' เป็น 'Account'
    # เพื่อให้ชื่อคอลัมน์ของ s2 สอดคล้องกับ s1 สำหรับการรวมกันภายหลัง
    s2 = s2.rename({'Account.1': 'Account'}, axis=1)

    # รวมข้อมูลบัญชีผู้โอนและผู้รับที่เกี่ยวข้องกับฟอกเงินเข้าด้วยกัน
    suspicious = pd.concat([s1, s2], join='outer')

    # ลบบัญชีซ้ำออก เพื่อให้เหลือบัญชีที่น่าสงสัยแต่ละบัญชีเพียง 1 ครั้ง
    suspicious = suspicious.drop_duplicates()


    # เปลี่ยนชื่อคอลัมน์ของ ldf และ rdf ให้เป็นชื่อเดียวกัน (Bank, Account)
    # จากนั้นเราจะรวมบัญชีฝั่งผู้โอนและผู้รับทั้งหมดเข้าด้วยกัน
    ldf = ldf.rename({'From Bank': 'Bank'}, axis=1)
    rdf = rdf.rename({'Account.1': 'Account', 'To Bank': 'Bank'}, axis=1)

    # รวมตารางฝั่งผู้โอนและผู้รับทั้งหมดเป็นตารางเดียว (รวมทุกบัญชีที่เคยปรากฏในระบบ)
    df = pd.concat([ldf, rdf], join='outer')

    # ลบบัญชีซ้ำ (เช่น บัญชีที่เคยเป็นทั้งผู้โอนและผู้รับ)
    df = df.drop_duplicates()


    #  เริ่มต้นกำหนดคอลัมน์ 'Is Laundering' ให้ทุกบัญชีเป็น 0 (ถือว่าปกติ)
    df['Is Laundering'] = 0

    # ตั้ง 'Account' เป็น index เพื่อให้ update ข้อมูลตามชื่อบัญชีได้ง่าย
    df.set_index('Account', inplace=True)

    #  อัปเดตค่าของบัญชีที่น่าสงสัย (จาก suspicious DataFrame)
    # โดยจะเขียนทับเฉพาะบัญชีที่อยู่ในรายการฟอกเงิน (Is Laundering = 1)
    df.update(suspicious.set_index('Account'))

    # รีเซ็ต index กลับมาเป็นคอลัมน์ปกติ
    df = df.reset_index()

    return df


Take a look of the account list:

In [36]:
accounts = get_all_account(df)
print(accounts.head())

       Account  Bank  Is Laundering
0  0_800060CE0     0              0
1  0_800061260     0              0
2  0_800062D90     0              0
3  0_800062F80     0              0
4  0_800064980     0              0


# Node features
For node features, we would like to aggregate the mean of paid and received amount with different types of currency as the new features of each node. 

In [ ]:
def paid_currency_aggregate(currency_ls, paying_df, accounts):
    """
    ฟังก์ชันนี้มีหน้าที่คำนวณค่าเฉลี่ยของจำนวนเงินที่ 'จ่ายออก' (Amount Paid)
    แยกตามแต่ละสกุลเงิน (Payment Currency)
    แล้วเพิ่มเป็นคอลัมน์ใหม่ให้กับ DataFrame ของบัญชี (accounts)
    """
    # วนลูปผ่านทุกสกุลเงินในรายการ
    for i in currency_ls:
        # เลือกเฉพาะธุรกรรมที่มีสกุลเงินตรงกับค่าปัจจุบัน i
        temp = paying_df[paying_df['Payment Currency'] == i]

        # คำนวณค่าเฉลี่ยของ Amount Paid ต่อบัญชี (Account)
        # แล้วเพิ่มผลลัพธ์เป็นคอลัมน์ใหม่ใน accounts
        accounts['avg paid ' + str(i)] = temp['Amount Paid'].groupby(temp['Account']).transform('mean')
    
    return accounts


def received_currency_aggregate(currency_ls, receiving_df, accounts):
    """
    ฟังก์ชันนี้มีหน้าที่คำนวณค่าเฉลี่ยของจำนวนเงินที่ 'ได้รับ' (Amount Received)
    แยกตามแต่ละสกุลเงิน (Receiving Currency)
    แล้วเพิ่มเป็นคอลัมน์ใหม่ให้กับ DataFrame ของบัญชี (accounts)
    """
    # วนลูปผ่านทุกสกุลเงินในรายการ
    for i in currency_ls:
        # เลือกเฉพาะธุรกรรมที่มีสกุลเงินตรงกับค่าปัจจุบัน i
        temp = receiving_df[receiving_df['Receiving Currency'] == i]

        # คำนวณค่าเฉลี่ยของ Amount Received ต่อบัญชี (Account)
        # แล้วเพิ่มผลลัพธ์เป็นคอลัมน์ใหม่ใน accounts
        accounts['avg received ' + str(i)] = temp['Amount Received'].groupby(temp['Account']).transform('mean')
    
    # เติมค่าที่หายไป (NaN) ด้วย 0 เพื่อป้องกันปัญหาขณะนำไปใช้กับโมเดล
    accounts = accounts.fillna(0)

    return accounts


Now we can define the node attributes by the bank code and the mean of paid and received amount with different types of currency.

In [ ]:
def get_node_attr(currency_ls, paying_df, receiving_df, accounts):
    """
    ฟังก์ชันนี้มีหน้าที่สร้าง "คุณลักษณะของโหนด (Node Attributes)" 
    และ "ป้ายกำกับของโหนด (Node Labels)" สำหรับแต่ละบัญชี
    เพื่อนำไปใช้เป็นข้อมูลป้อนเข้าโมเดลกราฟ (Graph Neural Network)
    """

    # ขั้นตอนที่ 1: รวมฟีเจอร์การจ่ายเงินของแต่ละบัญชีตามสกุลเงิน
    node_df = paid_currency_aggregate(currency_ls, paying_df, accounts)

    # ขั้นตอนที่ 2: รวมฟีเจอร์การรับเงินของแต่ละบัญชีตามสกุลเงิน
    node_df = received_currency_aggregate(currency_ls, receiving_df, node_df)

    # ขั้นตอนที่ 3: แปลงค่าป้ายกำกับ (Is Laundering) ให้เป็น tensor ของ PyTorch
    # ใช้ torch.float เพื่อให้สามารถใช้กับฟังก์ชัน Loss ได้
    node_label = torch.from_numpy(node_df['Is Laundering'].values).to(torch.float)

    # ขั้นตอนที่ 4: ลบคอลัมน์ที่ไม่ต้องใช้ในการเทรนโมเดลออก (Account, Is Laundering)
    node_df = node_df.drop(['Account', 'Is Laundering'], axis=1)

    # ขั้นตอนที่ 5: เข้ารหัส (Label Encoding) คอลัมน์ 'Bank' 
    # เพื่อให้รหัสธนาคารเป็นค่าตัวเลขที่โมเดลเข้าใจได้
    node_df = df_label_encoder(node_df, ['Bank'])

    # node_df = torch.from_numpy(node_df.values).to(torch.float)

    return node_df, node_label


Take a look of node_df:

In [39]:
node_df, node_label = get_node_attr(currency_ls, paying_df,receiving_df, accounts)
print(node_df.head())

   Bank  avg paid 0  avg paid 1  avg paid 2  avg paid 3  avg paid 4  \
0     0         0.0         0.0         0.0         0.0         0.0   
1     0         0.0         0.0         0.0         0.0         0.0   
2     0         0.0         0.0         0.0         0.0         0.0   
3     0         0.0         0.0         0.0         0.0         0.0   
4     0         0.0         0.0         0.0         0.0         0.0   

   avg paid 5  avg paid 6  avg paid 7  avg paid 8  avg paid 9  avg paid 10  \
0         0.0         0.0         0.0         0.0         0.0          0.0   
1         0.0         0.0         0.0         0.0         0.0          0.0   
2         0.0         0.0         0.0         0.0         0.0          0.0   
3         0.0         0.0         0.0         0.0         0.0          0.0   
4         0.0         0.0         0.0         0.0         0.0          0.0   

   avg paid 11    avg paid 12  avg paid 13  avg paid 14  avg received 0  \
0          0.0  307628.336486

# Edge features
In terms of edge features, we would like to conside each transcation as edges.  
For edge index, we replace all account with index and stack into a list with size of [2, num of transcation]  
For edge attributes, we used 'Timestamp', 'Amount Received', 'Receiving Currency', 'Amount Paid', 'Payment Currency' and 'Payment Format'


In [ ]:
def get_edge_df(accounts, df):
    """
    ฟังก์ชันนี้มีหน้าที่สร้างข้อมูลของ "ขอบของกราฟ (edges)" 
    สำหรับโมเดลกราฟ (Graph Neural Network) โดยแปลงข้อมูลธุรกรรมแต่ละรายการ 
    ให้กลายเป็นขอบระหว่างโหนดต้นทาง (ผู้โอน) และโหนดปลายทาง (ผู้รับ)
    """

    # ขั้นตอนที่ 1: รีเซ็ต index ของ accounts เพื่อให้แน่ใจว่า index เรียงลำดับใหม่จาก 0
    accounts = accounts.reset_index(drop=True)

    # ขั้นตอนที่ 2: สร้างคอลัมน์ 'ID' โดยใช้ค่า index เป็นรหัสของแต่ละบัญชี
    accounts['ID'] = accounts.index

    # ขั้นตอนที่ 3: สร้างพจนานุกรม (dictionary) สำหรับ map จากชื่อบัญชี → หมายเลข ID
    mapping_dict = dict(zip(accounts['Account'], accounts['ID']))

    # ขั้นตอนที่ 4: ใช้ mapping_dict เพื่อแปลงชื่อบัญชีในธุรกรรม
    # ให้กลายเป็นหมายเลข ID ของบัญชีต้นทาง (From) และปลายทาง (To)
    df['From'] = df['Account'].map(mapping_dict)
    df['To'] = df['Account.1'].map(mapping_dict)

    # ขั้นตอนที่ 5: ลบคอลัมน์ที่ไม่จำเป็นสำหรับการสร้างกราฟออก
    # (เช่น Account ชื่อจริง, รหัสธนาคารต้นทาง/ปลายทาง)
    df = df.drop(['Account', 'Account.1', 'From Bank', 'To Bank'], axis=1)

    # ขั้นตอนที่ 6: สร้าง tensor edge_index ที่เก็บคู่ (from, to) ของแต่ละธุรกรรม
    # shape = [2, จำนวนธุรกรรมทั้งหมด]
    edge_index = torch.stack([
        torch.from_numpy(df['From'].values),
        torch.from_numpy(df['To'].values)
    ], dim=0)

    # ขั้นตอนที่ 7: ลบคอลัมน์ที่ไม่ต้องการใน edge attribute
    # เช่น Is Laundering (label ของโหนด), และ index mapping ชั่วคราว
    df = df.drop(['Is Laundering', 'From', 'To'], axis=1)

    # edge_attr = torch.from_numpy(df.values).to(torch.float)

    # ใช้ DataFrame ดิบเป็น edge_attr แทน (เพื่อดูค่าได้ง่ายขณะทดลอง)
    edge_attr = df

    return edge_attr, edge_index


edge_attr:

In [41]:
edge_attr, edge_index = get_edge_df(accounts, df)
print(edge_attr.head())

         Timestamp  Amount Received  Receiving Currency  Amount Paid  \
3408783   0.266147          8081.58                   4      8081.58   
3986981   0.318925         47468.31                   4     47468.31   
4804475   0.393400          8081.58                   4      8081.58   
4804474   0.394151         47468.31                   4     47468.31   
6690464   0.547730           787.72                   4       787.72   

         Payment Currency  Payment Format  
3408783                 4               4  
3986981                 4               3  
4804475                 4               4  
4804474                 4               3  
6690464                 4               3  


edge_index:

In [42]:
print(edge_index)

tensor([[     0,      0,      0,  ..., 681281, 681282, 681282],
        [ 22343,  22343,  22343,  ..., 681281, 681282, 681282]])


# Final code 
### Below we will show the final code for model.py, train.py and dataset.py

# Model Architecture
In this section, we used Graph Attention Networks as our backbone model.  
The model built with two GATConv layers followed by a linear layer with sigmoid outout for classification

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv, Linear

class GAT(torch.nn.Module):
    """
    คลาสนี้เป็นการนิยามโมเดล Graph Attention Network (GAT)
    สำหรับงานจำแนกโหนด (Node Classification) เพื่อระบุว่าบัญชีใดมีพฤติกรรมฟอกเงินหรือไม่

    โครงสร้างโมเดล:
        - GATConv ชั้นที่ 1: เรียนรู้ความสัมพันธ์ระหว่างโหนด (ด้วย attention)
        - GATConv ชั้นที่ 2: ย่อยข้อมูลที่ผ่านการเรียนรู้จากชั้นแรก
        - Linear: แปลงข้อมูลที่ได้เป็นผลลัพธ์สำหรับการจำแนก
        - Sigmoid: ทำให้ออกมาเป็นค่าความน่าจะเป็น (0–1)
    """

    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        # เรียกใช้ constructor ของคลาสแม่ (torch.nn.Module)
        super().__init__()

        # ชั้นแรกของ GAT: ใช้ attention หลายหัว (multi-head attention)
        # เพื่อให้โมเดลเรียนรู้การเชื่อมโยงระหว่างโหนดที่ซับซ้อนได้ดีขึ้น
        self.conv1 = GATConv(
            in_channels,       # จำนวนฟีเจอร์อินพุตของโหนด
            hidden_channels,   # จำนวนฟีเจอร์เอาต์พุตหลังชั้นนี้
            heads,             # จำนวน attention heads
            dropout=0.6        # dropout ระหว่างการเรียนรู้
        )

        # ชั้นที่สองของ GAT: ลดขนาดของฟีเจอร์ (hidden_channels/4)
        # concat=False หมายความว่า output จากหลาย heads จะถูกเฉลี่ยรวมกัน
        self.conv2 = GATConv(
            hidden_channels * heads,  # input = output จากชั้นแรก (รวมทุก head)
            int(hidden_channels / 4), # ลดมิติลงเพื่อป้องกัน overfitting
            heads=1,                  # ใช้ head เดียวพอในชั้นที่สอง
            concat=False,             # ไม่ต่อผลของหลายหัวเข้าด้วยกัน
            dropout=0.6
        )

        # เลเยอร์ Linear: แปลงฟีเจอร์จาก GATConv สุดท้ายให้เป็นขนาดที่ต้องการ (out_channels)
        self.lin = Linear(int(hidden_channels / 4), out_channels)

        # Sigmoid: แปลงค่าเอาต์พุตสุดท้ายให้อยู่ในช่วง [0, 1] (เหมาะกับ binary classification)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, edge_index, edge_attr):
        """
        ฟังก์ชัน forward() คือการกำหนดลำดับขั้นตอนการไหลของข้อมูลในโมเดล
        โดยรับข้อมูลของกราฟแล้วส่งผ่านแต่ละเลเยอร์
        """

        # ขั้นตอนที่ 1: Dropout เพื่อป้องกัน overfitting
        x = F.dropout(x, p=0.6, training=self.training)

        # ขั้นตอนที่ 2: ผ่าน GATConv ชั้นที่ 1 พร้อมฟังก์ชันกระตุ้น ELU
        x = F.elu(self.conv1(x, edge_index, edge_attr))

        # ขั้นตอนที่ 3: Dropout อีกครั้งเพื่อเพิ่มความทนต่อ overfitting
        x = F.dropout(x, p=0.6, training=self.training)

        # ขั้นตอนที่ 4: ผ่าน GATConv ชั้นที่ 2 พร้อมฟังก์ชัน ELU
        x = F.elu(self.conv2(x, edge_index, edge_attr))

        # ขั้นตอนที่ 5: ผ่านเลเยอร์ Linear เพื่อให้ได้ขนาดเอาต์พุตสุดท้าย
        x = self.lin(x)

        # ขั้นตอนที่ 6: ใช้ Sigmoid เพื่อแปลงให้อยู่ในช่วง [0, 1]
        x = self.sigmoid(x)

        # คืนค่าผลลัพธ์สุดท้าย
        return x


# PyG InMemoryDataset
Finally we can build the dataset with above functions

In [ ]:
class AMLtoGraph(InMemoryDataset):
    """
    คลาสนี้ใช้สำหรับสร้างชุดข้อมูลกราฟ (Graph Dataset)
    จากข้อมูลธุรกรรมทางการเงิน เพื่อนำไปใช้เทรนโมเดลตรวจจับการฟอกเงิน (GNN)

    การทำงานหลักของคลาสนี้:
        1. โหลดข้อมูล CSV ดิบ
        2. ทำการแปลงและเตรียมข้อมูล (Preprocess)
        3. สร้างโหนด (Nodes) จากบัญชีทั้งหมด
        4. สร้างขอบ (Edges) จากธุรกรรมแต่ละรายการ
        5. สร้างฟีเจอร์ของโหนดและขอบ (Attributes)
        6. รวมทั้งหมดเป็นอ็อบเจกต์ของ PyTorch Geometric Data
    """

    def __init__(self, root: str, edge_window_size: int = 10,
                 transform: Optional[Callable] = None,
                 pre_transform: Optional[Callable] = None):
        # บันทึกค่า edge_window_size ไว้ในคลาส
        self.edge_window_size = edge_window_size

        # เรียกใช้งาน constructor ของคลาสแม่ InMemoryDataset
        super().__init__(root, transform, pre_transform)

        # โหลดข้อมูลที่ประมวลผลไว้แล้ว (processed data) จากไฟล์ data.pt
        self.data, self.slices = torch.load(self.processed_paths[0], weights_only=False)

    # ------------------------------------------------------------
    # ส่วนกำหนดชื่อไฟล์ที่ใช้ (สำหรับระบบของ PyTorch Geometric)
    # ------------------------------------------------------------
    @property
    def raw_file_names(self) -> str:
        # ชื่อไฟล์ข้อมูลต้นฉบับ (ต้องอยู่ในโฟลเดอร์ raw/)
        return 'LI-Small_Trans.csv'

    @property
    def processed_file_names(self) -> str:
        # ชื่อไฟล์ที่เก็บข้อมูลหลังประมวลผลเสร็จแล้ว
        return 'data.pt'

    @property
    def num_nodes(self) -> int:
        # จำนวนโหนดทั้งหมดในกราฟ
        # โดยดูจากหมายเลขโหนดสูงสุดใน edge_index + 1
        return self._data.edge_index.max().item() + 1

    # ------------------------------------------------------------
    # ฟังก์ชันย่อย: เข้ารหัสข้อมูลเชิงหมวดหมู่เป็นตัวเลข
    # ------------------------------------------------------------
    def df_label_encoder(self, df, columns):
        le = preprocessing.LabelEncoder()
        for i in columns:
            df[i] = le.fit_transform(df[i].astype(str))
        return df

    # ------------------------------------------------------------
    # ฟังก์ชันย่อย: เตรียมข้อมูล (Preprocessing)
    # ------------------------------------------------------------
    def preprocess(self, df):
        # เข้ารหัสคอลัมน์เชิงหมวดหมู่ (เช่น สกุลเงิน, รูปแบบการชำระเงิน)
        df = self.df_label_encoder(df, ['Payment Format', 'Payment Currency', 'Receiving Currency'])

        # แปลงเวลาเป็น datetime และ normalize ให้อยู่ในช่วง [0, 1]
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df['Timestamp'] = df['Timestamp'].apply(lambda x: x.value)
        df['Timestamp'] = (df['Timestamp'] - df['Timestamp'].min()) / (df['Timestamp'].max() - df['Timestamp'].min())

        # รวมรหัสธนาคารกับหมายเลขบัญชี เพื่อสร้างรหัสบัญชีไม่ซ้ำ
        df['Account'] = df['From Bank'].astype(str) + '_' + df['Account']
        df['Account.1'] = df['To Bank'].astype(str) + '_' + df['Account.1']

        # เรียงข้อมูลตามชื่อบัญชีเพื่อความสม่ำเสมอ
        df = df.sort_values(by=['Account'])

        # แยกข้อมูลฝั่งผู้รับและผู้จ่ายออกมา
        receiving_df = df[['Account.1', 'Amount Received', 'Receiving Currency']]
        paying_df = df[['Account', 'Amount Paid', 'Payment Currency']]

        # เปลี่ยนชื่อคอลัมน์ Account.1 → Account เพื่อให้ชื่อสอดคล้องกัน
        receiving_df = receiving_df.rename({'Account.1': 'Account'}, axis=1)

        # เก็บรายการรหัสสกุลเงินทั้งหมด
        currency_ls = sorted(df['Receiving Currency'].unique())

        return df, receiving_df, paying_df, currency_ls

    # ------------------------------------------------------------
    # ฟังก์ชันย่อย: สร้างตารางบัญชีทั้งหมด (Nodes)
    # ------------------------------------------------------------
    def get_all_account(self, df):
        # ดึงข้อมูลบัญชีจากฝั่งผู้โอนและผู้รับ
        ldf = df[['Account', 'From Bank']]
        rdf = df[['Account.1', 'To Bank']]

        # ดึงธุรกรรมที่ถูกระบุว่าฟอกเงิน
        suspicious = df[df['Is Laundering'] == 1]
        s1 = suspicious[['Account', 'Is Laundering']]
        s2 = suspicious[['Account.1', 'Is Laundering']]
        s2 = s2.rename({'Account.1': 'Account'}, axis=1)
        suspicious = pd.concat([s1, s2], join='outer').drop_duplicates()

        # รวมข้อมูลผู้โอนและผู้รับทั้งหมด
        ldf = ldf.rename({'From Bank': 'Bank'}, axis=1)
        rdf = rdf.rename({'Account.1': 'Account', 'To Bank': 'Bank'}, axis=1)
        df = pd.concat([ldf, rdf], join='outer').drop_duplicates()

        # เพิ่มคอลัมน์ Is Laundering เริ่มต้นเป็น 0
        df['Is Laundering'] = 0

        # อัปเดตค่าของบัญชีที่เกี่ยวข้องกับการฟอกเงินให้เป็น 1
        df.set_index('Account', inplace=True)
        df.update(suspicious.set_index('Account'))
        df = df.reset_index()
        return df

    # ------------------------------------------------------------
    # ฟังก์ชันย่อย: คำนวณค่าเฉลี่ยการจ่ายเงินแยกตามสกุลเงิน
    # ------------------------------------------------------------
    def paid_currency_aggregate(self, currency_ls, paying_df, accounts):
        for i in currency_ls:
            temp = paying_df[paying_df['Payment Currency'] == i]
            accounts['avg paid ' + str(i)] = temp['Amount Paid'].groupby(temp['Account']).transform('mean')
        return accounts

    # ------------------------------------------------------------
    # ฟังก์ชันย่อย: คำนวณค่าเฉลี่ยการรับเงินแยกตามสกุลเงิน
    # ------------------------------------------------------------
    def received_currency_aggregate(self, currency_ls, receiving_df, accounts):
        for i in currency_ls:
            temp = receiving_df[receiving_df['Receiving Currency'] == i]
            accounts['avg received ' + str(i)] = temp['Amount Received'].groupby(temp['Account']).transform('mean')
        accounts = accounts.fillna(0)
        return accounts

    # ------------------------------------------------------------
    # ฟังก์ชันย่อย: สร้างข้อมูลขอบ (Edges)
    # ------------------------------------------------------------
    def get_edge_df(self, accounts, df):
        accounts = accounts.reset_index(drop=True)
        accounts['ID'] = accounts.index
        mapping_dict = dict(zip(accounts['Account'], accounts['ID']))

        # แปลงชื่อบัญชีในธุรกรรมให้เป็นหมายเลข ID
        df['From'] = df['Account'].map(mapping_dict)
        df['To'] = df['Account.1'].map(mapping_dict)

        # ลบคอลัมน์ที่ไม่จำเป็น
        df = df.drop(['Account', 'Account.1', 'From Bank', 'To Bank'], axis=1)

        # สร้าง tensor edge_index ที่เก็บคู่ (from, to)
        edge_index = torch.stack([
            torch.from_numpy(df['From'].values),
            torch.from_numpy(df['To'].values)
        ], dim=0)

        # ลบคอลัมน์ที่ไม่ต้องการออกจาก edge attribute
        df = df.drop(['Is Laundering', 'From', 'To'], axis=1)

        # แปลงค่า edge attributes เป็น tensor ของตัวเลขแบบ float
        edge_attr = torch.from_numpy(df.values).to(torch.float)
        return edge_attr, edge_index

    # ------------------------------------------------------------
    # ฟังก์ชันย่อย: สร้างฟีเจอร์และ label ของโหนด
    # ------------------------------------------------------------
    def get_node_attr(self, currency_ls, paying_df, receiving_df, accounts):
        node_df = self.paid_currency_aggregate(currency_ls, paying_df, accounts)
        node_df = self.received_currency_aggregate(currency_ls, receiving_df, node_df)

        # Label ของโหนด (0 = ปกติ, 1 = ฟอกเงิน)
        node_label = torch.from_numpy(node_df['Is Laundering'].values).to(torch.float)

        # ลบคอลัมน์ที่ไม่ใช้
        node_df = node_df.drop(['Account', 'Is Laundering'], axis=1)

        # เข้ารหัสคอลัมน์ 'Bank' เป็นตัวเลข
        node_df = self.df_label_encoder(node_df, ['Bank'])

        # แปลงข้อมูลฟีเจอร์เป็น tensor
        node_df = torch.from_numpy(node_df.values).to(torch.float)
        return node_df, node_label

    # ------------------------------------------------------------
    # ฟังก์ชันหลัก: ประมวลผลข้อมูลและสร้างกราฟ
    # ------------------------------------------------------------
    def process(self):
        # โหลดข้อมูลจากไฟล์ CSV ดิบ
        df = pd.read_csv(self.raw_paths[0])

        # ทำการ preprocess และแยกข้อมูลออกเป็นส่วนต่าง ๆ
        df, receiving_df, paying_df, currency_ls = self.preprocess(df)

        # สร้างตารางบัญชีทั้งหมด (nodes)
        accounts = self.get_all_account(df)

        # สร้างคุณลักษณะของโหนดและป้ายกำกับ
        node_attr, node_label = self.get_node_attr(currency_ls, paying_df, receiving_df, accounts)

        # สร้างข้อมูลขอบ (edges)
        edge_attr, edge_index = self.get_edge_df(accounts, df)

        # สร้างอ็อบเจกต์ Data ของ PyTorch Geometric
        data = Data(
            x=node_attr,
            edge_index=edge_index,
            y=node_label,
            edge_attr=edge_attr
        )

        # จัดเก็บข้อมูลเป็น list เดียว (กรณีมีหลายกราฟ)
        data_list = [data]

        # ตรวจสอบ pre_filter (ถ้ามี)
        if self.pre_filter is not None:
            data_list = [d for d in data_list if self.pre_filter(d)]

        # ตรวจสอบ pre_transform (ถ้ามี)
        if self.pre_transform is not None:
            data_list = [self.pre_transform(d) for d in data_list]

        # รวมข้อมูลและบันทึกไฟล์ที่ประมวลผลแล้ว
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])


# Model Training 
As we cannot create folder in kaggle, please follow the instructions in https://github.com/issacchan26/AntiMoneyLaunderingDetectionWithGNN before you start training 

In [45]:
import importlib, dataset
importlib.reload(dataset)

<module 'dataset' from 'c:\\Users\\jirap\\OneDrive\\Desktop\\Gits\\Anti-Money-Laundering-Datamining\\GNN\\dataset.py'>

In [ ]:
import torch
import torch_geometric.transforms as T

# ------------------------------------------------------------
# 1) กำหนดอุปกรณ์ประมวลผล (Device)
# ------------------------------------------------------------
# ถ้ามี GPU จะใช้ 'cuda' ถ้าไม่มีก็ใช้ 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ------------------------------------------------------------
# 2) โหลดข้อมูลกราฟที่ถูกประมวลผลไว้แล้ว (จาก data.pt)
# ------------------------------------------------------------
# root='./data' คือโฟลเดอร์ที่มีไฟล์ processed/data.pt
dataset = AMLtoGraph('./data')

# ดึงกราฟแรกจาก dataset (กรณีมีเพียงกราฟเดียว)
data = dataset[0]

# ตั้งจำนวนรอบการเทรน (epochs)
epoch = 100

# ------------------------------------------------------------
# 3) สร้างโมเดล GAT และเตรียมฟังก์ชัน Loss + Optimizer
# ------------------------------------------------------------
# สร้างโมเดล GAT โดยกำหนดพารามิเตอร์:
# - in_channels: จำนวนฟีเจอร์ของโหนด (data.num_features)
# - hidden_channels: จำนวนหน่วยในเลเยอร์ซ่อน
# - out_channels: ขนาดเอาต์พุต (1 สำหรับ binary classification)
# - heads: จำนวน attention heads
model = GAT(in_channels=data.num_features, hidden_channels=16, out_channels=1, heads=8)

# ย้ายโมเดลไปยังอุปกรณ์ประมวลผล (GPU/CPU)
model = model.to(device)

# ฟังก์ชัน Loss ที่ใช้วัดข้อผิดพลาด (Binary Cross Entropy)
criterion = torch.nn.BCELoss()

# Optimizer ที่ใช้ปรับค่าน้ำหนัก (ใช้ SGD แบบ learning rate คงที่)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

# ------------------------------------------------------------
# 4) แบ่งข้อมูล Train / Validation / Test ด้วย RandomNodeSplit
# ------------------------------------------------------------
# - split='train_rest' หมายถึง train 80% + val 10% + test 10% (โดยประมาณ)
# - num_val=0.1 คือสัดส่วน validation = 10%
# - num_test=0 คือไม่มี test set แยกออกมา (ใช้ train/val เท่านั้น)
split = T.RandomNodeSplit(split='train_rest', num_val=0.1, num_test=0)

# ใช้การ split กับข้อมูลกราฟ
data = split(data)

# ย้ายข้อมูลกราฟทั้งหมดไปยังอุปกรณ์เดียวกับโมเดล
data = data.to(device)

# ------------------------------------------------------------
# 5) เริ่มกระบวนการเทรนโมเดล (Training Loop)
# ------------------------------------------------------------
for i in range(epoch):
    # ตั้งสถานะโมเดลให้เป็น train mode (เปิดใช้ dropout)
    model.train()

    # เคลียร์ gradient จากรอบก่อนหน้า
    optimizer.zero_grad()

    # ส่งข้อมูลกราฟเข้าโมเดล เพื่อให้โมเดลทำนายผล (prediction)
    pred = model(data.x, data.edge_index, data.edge_attr)

    # คำนวณค่า loss เฉพาะโหนดที่อยู่ใน train_mask
    # data.y มีขนาด [num_nodes] → ต้อง unsqueeze(1) ให้เป็น [num_nodes, 1]
    loss = criterion(pred[data.train_mask], data.y[data.train_mask].unsqueeze(1))

    # ทำการย้อนแพร่กระจายค่าความผิดพลาด (backpropagation)
    loss.backward()

    # ปรับค่าน้ำหนักของโมเดลตาม gradient
    optimizer.step()

    # --------------------------------------------------------
    # 6) ประเมินผลบน Validation Set ทุก ๆ 10 epoch
    # --------------------------------------------------------
    if (i + 1) % 10 == 0:
        # ตั้งสถานะเป็น eval mode (ปิด dropout)
        model.eval()

        with torch.no_grad():
            # ทำการทำนายอีกครั้งโดยไม่คำนวณ gradient
            val_pred = model(data.x, data.edge_index, data.edge_attr)

            # ดึงผลเฉพาะโหนดที่อยู่ใน validation mask
            val_probs = val_pred[data.val_mask]
            val_labels = data.y[data.val_mask].unsqueeze(1)

            # คำนวณค่า loss บน validation set
            val_loss = criterion(val_probs, val_labels)

            # แปลงค่าความน่าจะเป็น (>0.5 → 1, <=0.5 → 0)
            # แล้วคำนวณ accuracy
            val_acc = ((val_probs > 0.5).float() == val_labels).float().mean().item()

        # แสดงผลทุก 10 รอบ
        print(f"Epoch: {i + 1:03d}, "
              f"Loss: {loss.item():.4f}, "
              f"Val Loss: {val_loss.item():.4f}, "
              f"Val Acc: {val_acc:.4f}")


Epoch: 010, Loss: 28.9350, Val Loss: 36.0975, Val Acc: 0.6349
Epoch: 020, Loss: 28.2222, Val Loss: 35.2105, Val Acc: 0.6442
Epoch: 030, Loss: 28.2341, Val Loss: 34.6094, Val Acc: 0.6503
Epoch: 040, Loss: 27.6121, Val Loss: 33.7354, Val Acc: 0.6592
Epoch: 050, Loss: 30.7693, Val Loss: 32.8628, Val Acc: 0.6685
Epoch: 060, Loss: 26.5314, Val Loss: 32.0509, Val Acc: 0.6770
Epoch: 070, Loss: 27.8385, Val Loss: 28.6403, Val Acc: 0.7110
Epoch: 080, Loss: 26.2114, Val Loss: 28.2124, Val Acc: 0.7158
Epoch: 090, Loss: 26.7170, Val Loss: 27.6892, Val Acc: 0.7211
Epoch: 100, Loss: 26.6882, Val Loss: 27.2748, Val Acc: 0.7252


## Reference
Some of the feature engineering of this repo are referenced to below papers, highly recommend to read:
1. [Weber, M., Domeniconi, G., Chen, J., Weidele, D. K. I., Bellei, C., Robinson, T., & Leiserson, C. E. (2019). Anti-money laundering in bitcoin: Experimenting with graph convolutional networks for financial forensics. arXiv preprint arXiv:1908.02591.](https://arxiv.org/pdf/1908.02591.pdf)
2. [Johannessen, F., & Jullum, M. (2023). Finding Money Launderers Using Heterogeneous Graph Neural Networks. arXiv preprint arXiv:2307.13499.](https://arxiv.org/pdf/2307.13499.pdf)